Created on Tuesday 12th January 2021 

# **POC mesures de pertinence**
**Group 2 - Recherche de nouvelles sources**  
*Projet Inter-Promo 2021 de la formation SID, Université Paul Sabatier, Toulouse*

@authors : Corentin Prat-Marca, Nicolas Enjalbert Courrech, Sonia Bezombe, Flavien Caminade, Audrey Degaugue

Un point clef de la veille technologique automatisée est de définir des nouvelles sources répondant au thème de cette veille. 
Dans le cadre de ce projet, l'identification des nouvelles sources pertinentes s'est faite en plusieurs étapes. La première est de récupérer un nombre important d'articles. Pour cela, une solution utilisant un moteur de recherche et un ensemble d'équation. Ces points sont détaillés dans le POC ou rapport dédié. Le résultat de cette étape-ci est un ensemble d'articles répondant plus ou moins aux requêtes.  
L'étape suivante permet de mesurer la pertinence d'un article et par agrégation d'un site entier. Le but de cette étape est de définir si un site est qualifié de pertinent et le positionner dans la liste des sites à surveiller (ici en les scrappant) tous les jours.
Ce document décrit des mesures qui ont été pensées durant le projet. 

Ces mesures sont calculées à partir d'un data frame obtenu après les résultats de moteur de recherche contenant :  
  * art_url : l'url de l'article
  * src_name : le nom du site
  * src_url : l'url du site (page d'accueil)
  * query : mots clefs utilisés pour effectuer la recherche
  * title : titre proposé par le moteur de recherche
  * snippet : résumé proposé par le moteur de recherche
  * rank : position de l'article dans le résultat du moteur de recherche


## Mesure de popularité

La première mesure mise en place est très naïve : elle mesure la popularité d'un site, c'est-à-dire, le nombre de fois que le site est appartu à partir d'un ensemble de requêtes. 
Un simple *count* répond à cette question. 

In [ ]:
def countSite(df : pd.DataFrame):
  # Count the number of URLs crawled per site
  return df.groupby('src_name').count().reset_index()

def popularite(df : pd.DataFrame):
    """
    Calcule the popularity score per site

    Parameters
    ----------
    df : DataFrame
        Contains : art_url, src_name, src_url.

    Returns
    -------
    DataFrame
        Contains : src_name, popularity score.

    """
    df1 = countSite(df)
    df1["popularity"] = [x/np.max(df1.art_url) for x in df1.art_url]
    df_final = df1[['src_name','popularity']]
    return df_final

Cette mesure ne répond pas complètement à la question de pertinence. Les moteurs de recherche retournent souvent en priorité une liste de sites populaires, car très utilisés par les utilisateurs des moteurs de recherche, comme des réseaux sociaux, dictionnaires, traducteurs ou encyclopédies. Ces sites nous ont été retournés lors de notre première mise en place de récupération des résultats de moteurs de recherche. 

Cet ensemble de sites ne répondent pas à notre recherche d'articles innovants et très actuels. De plus, il ne prend pas en compte de site moins populaire mais contenant des articles très pertinents. Cette solution naïve n'est donc pas judicieuse. 


 ## Mesure du rang

Cette mesure s'obtient en se basant sur l'ordonnancement des articles par le moteur de recherche. Intuitivement, un article étant classé en première position répond mieux à la requête de recherche qu'un article étant placé en dernière position. 

La fonction _score_rank_ permet de mettre en valeur le rang de position en attribuant un score élevé à l'article étant au premier rang et en agrégeant par la moyenne des scores par sites. 

$scoreArticle_i = 1 - \frac{rang_i}{card(R)} $  avec R l'ensemble des résultats de la requête donnant l'article i. 

La fonction d'agrégation est la moyenne de ces scores groupés par sites $\frac{1}{card(c)} \sum_{c \in C} scoreArticle_c $


In [ ]:
def score_rank(df : pd.DataFrame) :
    """ Documentation

    Calculate the score rank per site based on the position of the item in the
    crawl 

    Parameters :
        df : dataframe processed
    Out :
        df_result_rank : dataframe containing the score rank per site
    """

    for i in range(df.shape[0]):
        df['score_rank'] = 1/((df['position']/df[df['query'] == df['query'].iloc[i]].shape[0])+1) # the rank score is calculated based on the position of the item in the crawl 
        df_result_rank = df.groupby('src_name')['score_rank'].mean() # we do a groupby to get the average of this score per site,
    return (df_result_rank)

Ce score est aussi biaisé par les sites populaires qui sont placés souvent en première position. 

## Mesure du contenu de la requête dans le titre et le résumé

Cette mesure vise à savoir si les mots de la requête se retrouvent en leur totalité ou non dans le titre et le résumé proposés par le moteur de recherche. 
Cette mesure propose un score compris entre 0 et 1 pour chaque site.

In [ ]:
def common_query_words(df : pd.DataFrame) :
    """ Documentation

    Calculate the relevance of the queries

    Parameters :
        df : dataframe processed
    Out :
        df : dataframe containing relevance score and categorie
    """

  df = transform_data(df)
  df['cat_pertinence'] = ''
  df['common_words'] = ''
  df['relevance_query']=0.00
  relevance_query : list = [] #list that will store the relevance score,
    #pertinence_j = [] #list that will store the number of the line concerned
  df_relevance = pd.DataFrame(columns=['nb_row','score']).set_index('nb_row') #creation of a df allowing to have the score for each line,
  for j in range (len(df)): # for each line of the df,
    innov : int = 0 #initialization of a variable to count the number of words in the innovation lexicon 
    gest : int = 0 #...the same for the management lexicon
    separator_or : list = list(df['query'][j].split(' or ')) #we store all pairs of the request in a separator_or list 
    relevance_listing : list = [] #list to store the relevance scores for each of the couples
    list_find : list = [] #list to store words in common for each line,
    for k in range (len(separator_or)): #for each of these couples,
      nb_present : int = 0 #count the number of words in common
      separator_and : list = list(separator_or[k].split(' and ')) #we store all the words of the couple of the query in a separator_and list 
      for i in range (len(separator_and)): #for all the words in the query,
        if (df['title'][j].find(separator_and[i]) != -1) : #we look for it in the title, and if it's there... 
          if (i == 0): #if it's the first member of the couple, we've found a word of innovation 
            innov += 1
          else : #If not, a word of management 
            gest += 1
          nb_present = nb_present + 1 #increments the number of words in the query found.  
          list_find.append(separator_and[i]+";")
        if (df['resume'][j].find(separator_and[i]) != -1) : #then we look for it in the summary, and if it's there... 
          if (i == 0):
            innov += 1
          else :
            gest += 1
          nb_present += 1
          list_find.append(separator_and[i] +";") #... we put it in a list
          list_find = list(set(list_find))
      relevance_query : float = (nb_present / len(separator_and))*100 #we calculate the relevance score nb of words found / nb of total words in the query
      relevance_listing.append(relevance_query)
      str = ' '.join(list_find) #...
    df['common_words'][j]=str[:-1] #...we add the words found in the df
    df_relevance = df_relevance.append({'nb_row': j}, ignore_index=True)
    df_relevance['score'][j] = max(relevance_listing) #for each line, we take the best relevance of a couple,
    df['relevance_query'][j] = df_relevance['score'][j]
    nb = df_relevance['score'][j]
    #and we categorize according to the score obtained :
    if (nb>=100) : #all the words of at least one couple are found,
      df['cat_pertinence'][j] = 'I&G'
    else :
      if (innov >= 1 and gest >= 1) : #words of innovation and gestion are found but not in the same couple,
        df['cat_pertinence'][j] = 'I&G but not from the same couple'
      elif (innov >= 1 and gest == 0) : #at least one word of innovation is found, but no gestion,
        df['cat_pertinence'][j] = 'I'
      elif (innov == 0 and gest >= 1): #at least one word of management is found but no innovation, 
        df['cat_pertinence'][j] = 'G'
      else :
        df['cat_pertinence'][j] = 'None' #no word is found 
  return(df)

def result_score_def (df : pd.DataFrame):
    """ Documentation

    Calculate the relevance of a query per site

    Parameters :
        df : dataframe processed
    Out :
        df_result_score : dataframe containing the relevance of a query per
        site
    """

    df_relevance_query = common_query_words(df) # record our relevance scores obtained using common words
    df_relevance_query['src_name'] = ''
    for i in range (len(df_relevance_query)) :
        df_relevance_query['src_name'][i] = site_name(df_relevance_query['art_url'][i]) # add the name of the site in the dataframe according to the url 
    df_result_score = df_relevance_query.groupby('src_name')['relevance_query'].mean() # we make a groupby to have the average score according to the request per site 
    return (df_result_score) # return the new dataframe  

def relevance_query (df : pd.DataFrame) :
    """ Documentation

    Calculate the standardized relevance of a query per site

    Parameters :
        df : dataframe processed
    Out :
        df_result_score : dataframe containing the relevance of a query per
        site
    """

    df_result_score = result_score_def(df) # we record our relevance scores grouped by site, 
    df_result_score = df_result_score.reset_index()
    df_result_score['relevance_query'] = df_result_score['relevance_query']/(max(df_result_score['relevance_query'])) # they are divided by the maximum to get a relevance score between 0 and 1,
    return (df_result_score)

Cette mesure prend en compte les mots de la requête étant dans le titre et le résumé. Dans notre cas, le moteur de recherche Google, choisi pour ce projet, utilise automatiquement aussi des synonymes des mots contenus dans la requête textuelle. Cette approche prend en compte des formes réduites des mots pouvant ainsi prendre en compte toutes les formes de notre recherche textuelle. 

Néanmoins, un synonyme ne répond pas à ces formes réduites puisque différents par leur racine. Cette mesure ne prend donc pas en compte un potentiel article très pertinent mais ne répondant qu'à des synonymes de la requête textuelle utilisée. 


## Mesure : présence des mots du lexique dans l'article.

Une première approche pour répondre à la problématique des synonymes est d'utiliser l'ensemble des mots clefs des lexiques proposés en entrée des requêtes textuelles. 

Les fonctions suivantes permettent de calculer ce score-ci.

On cherche à attribuer un score à des articles scrappés en regardant combien on trouve de mots de chaque lexique. On choisit sur quelles parties on se base (nom de l'article, résumé, contenu, etc), et on calcule un score dépendant du nombre de mots des lexiques (plus il y a de mots mieux c'est, si un article comporte 0 mots d'un lexique on pénalise) de la différence des nombres de mots de chaque lexique (on veut à peu près autant de mots de chaque lexique), et de la taille de l'article.

In [1]:
import nltk   #the importations for cleandesc
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer 
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('french') #we're working on french articles
import pandas as pd
import re

def cleandesc(desc : str): #function to remove stopwords and suffixes
    """Documentation
    Function took from @SoniaBezombes
    """
    sent = desc
    sent = "".join([x.lower() if x.isalpha()  else " " for x in sent])
    Porter=SnowballStemmer('french')
    sent = " ".join([Porter.stem(x) if x.lower() not in stop_words  else "" for x in sent.split()])
    sent = " ".join(sent.split())
    return sent

def calcul_score(texte : str,lexique : list): 
    #function to count the number of words of a list in a text
    """Documentation
    Parameters:
        texte : the text we're working on
        lexique : the list of words we're looking for
    Out :
        somme : the number of lexique's words we found in texte
    """
    somme : int = 0
    for mot in lexique:
        somme += texte.count(' '+mot+' ')
    return somme


#before using pertinence 4, we have to clean the columns we're gonna work on like that :
#df_scrapped_articles['column_name']= [cleandesc(x.column_name) for x in df_scrapped_articles.itertuples()]
#cleaning art_content all the time, just once
#df_scrapped_articles['art_content']= [cleandesc(x.art_content) for x in df_work.itertuples()]

def pertinence_4(df : pd.DataFrame , list_columns : list, 
                 lexiqueI : list, lexiqueG : list):
    """Documentation
    Parameters:
        df : a data frame of scrapped articles. It must contains 
        a "art_content" columns to determine its size
        
        list_columns : the list of columns we want to count 
        the words of the two lexiques on. 
        
        lexiqueI : the innovation lexique, it has to be a list
        lexiqueG : the gestion lexique, it has to be a list
    Out :
        df : the same data but with the scores and the step to have it
    """
    list_nom_colonne_I : list = [] #the list of the names of the innovation's score columns 
    list_nom_colonne_G : list = [] #the list of the names of the gestion's score columns 
    cleanedG : list = [cleandesc(mot) for mot in lexiqueG] #cleaning the gestion's lexique
    cleanedI : list = [cleandesc(mot) for mot in lexiqueI] #cleaning the gestion's lexique

    for columns in list_columns:

        nomI : str ='score_innovation_'+columns #the name of the new columns : score_innovation_column_name or score_gestion_column_name
        nomG : str ='score_gestion_'+columns
        df[nomI]=[calcul_score(df.loc[i][columns], cleanedI) for i in range(len(df))] #creating the new columns using calcul_score, counting the number of words
        df[nomG]=[calcul_score(df.loc[i][columns], cleanedG) for i in range(len(df))]
        df['score_'+columns]=[df.loc[i][nomI]+df.loc[i][nomG] for i in range(len(df))] #new column, wich is the sum of gestion and innovation scores
        list_nom_colonne_I.append(nomI)
        list_nom_colonne_G.append(nomG)
    
    #length of the cleaned content of the article
    df['taille_content'] = [len(df['art_content'][i]) for i in range(len(df))] 
    #sum of all the innovation's scores
    df['score_I'] = df[list_nom_colonne_I].sum(axis = 1) 
    #sum of all the gestion's scores
    df['score_G'] = df[list_nom_colonne_G].sum(axis = 1) 
    #absolute value of the difference of the number of gestion's words and innovation's words
    df['score_abs'] = abs(df['score_I']-df['score_G'])
    #finding if there is 0 gestion's words or 0 innovation's words, 
    #to penalise the articles which don't speak of innovation or gestion
    df['detct_0'] = [0 if (x.score_I == 0) or (x.score_G == 0) else 1 for x in df.itertuples()] 
    
    df['score_total'] = [(1.09*(x.score_I+x.score_G)-abs(x.score_I-x.score_G))*100/x.taille_content 
                         if x.detct_0 == 0 
                         else (1.09*(x.score_I+x.score_G)-abs(x.score_I-x.score_G))*1000/\
                         x.taille_content 
                         for x in df.itertuples()]

    return df

[nltk_data] Downloading package stopwords to /home/cmisid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Obtenant ainsi un score par article, la fonction d'agrégation utilisée est la moyenne des scores par sites webs.
Pour cela on utilise une fonction qui, à partir d'un url d'article, trouve l'url du site source. Du coup, on se retrouve avec linkedin.com =/= fr.linkedin, mais 2 sites différents avec le même nom (site.fr, www.site.com, etc) sont bien différenciés. Cela nous permet aussi de faire la différence entre la version française et anglaise d'un site.

In [ ]:
def url_site(url_art : str):
    """Documentation
    @Flavien Caminade et Corentin Prat-Marca
    Parameters:
        url_art : the url of an article
    Out :
        url_site : the url of a site obtained with the article's url
    """
    site : list  =url_art.split("://")
    #keep everything on the right side of "https://""
    if site[0]=="https" or site[0]=="http":
        url_site : str = site[1]
    else:
        url_site : str = site[0]
    #delete everything after the first "/"
    tab : list = url_site.split("/")
    url_site = tab[0]
    return url_site

def pertinence_4_source(df_pertinence_4 : pd.DataFrame):
    """Documentation
    Parameters:
        df_pertinence_4 : a data frame of scrapped articles with their score
    Out :
        df : a data frame with the score for each sites
    """
    dico : dict ={} #dictionnary : keys=sources url, values=sum of the scores of each articles
    for articles in df_calcul_global.itertuples():

        if url_site(articles.art_url) in dico: #if the source il already in the dictionnary
            dico[url_site(articles.art_url)][0] += (articles.score_total)**2 #using square so higher score have more values
            dico[url_site(articles.art_url)][1] += 1 #increasing the number of articles for the source
        else :
            dico[url_site(articles.art_url)] = [(articles.score_total)**2, 1]

    df : pd.DataFrame = pd.DataFrame.from_dict(dico, orient = 'index', columns = ['score_total', 'nb_articles'])
    df['score_moyen'] = df.apply(lambda row: row.score_total / row.nb_articles, axis = 1) #mean of the scores of the artciles for each sources
    return df
#pertinence_4_source(df_pertinence_4)

Cette mesure apporte une meilleure pertinence si elle est utilisée sur le contenu de l'article. Or cette solution fait appel à l'obligation de scrapper les articles pour récupérer le contenu. 
Or, en suivant les problématiques des scrapeurs (cf documents spécifiques) on se rend compte que le scrapeur générique est nécessaire pour récupérer le contenu des articles. Cette hypothèse forte rend l'application de cette mesure difficile à mettre en place. 

Néanmoins, en partant du principe que le scrapeur générique est fonctionnel et performant, cette mesure de pertinence permet de prendre en compte les synonymes soulevés dans la mesure précédente. 


## Mesure : utilisation des résultats de modélisation en classification (résultat du groupe 5)

Une autre solution pour prendre en compte les synonymes et bien définir le sujet des articles est d'utiliser les résultats du groupe 5 du projet. Ce groupe a pour objectif de faire une classification et définir si un article parle d'innovation, de gestion ou d'un autre thème. Deux de leur résultat sont de donner la probabilité qu'un article parle d'innovation et une probabilité qu'un article parle de gestion. 

Le data frame d'entrée de ces fonctions sont : 
  * art_url : l'url de l'article
  * src_name : le nom du site web
  * src_url : l'url du site web correspondant à sa page d'accueil
  * probaInnovation : la probabilité que l'article parle d'innovation $[0;1]$
  * probaGestion : la probabilité que l'article parle de gamme de gestion $[0;1]$
  
  
Pour calculer la probabilité conjointe, nous avons fait plusieurs mesures d'agrégation des probabilité. Le but est de prendre en compte qu'un article parle à la fois d'innovation et de gestion. Ainsi un article qui parle beaucoup de gestion mais pas d'innovation devra être moins bien classé qu'un article parlant moyennement d'innovation et de gestion (ici proba = 0.5 pour gestion et innovation). 


Cette fonction permet de créer un score de pertinence à partir d'un data frame d'articles scrappés dont on a les probabilités d'apparitions d'un mot de gestion ou d'innovation (travail du groupe 5).
Elle permet d'évaluer les articles pour avoir une idée de la pertinence. Les scores vont de 0 à 1, et prennent en compte la somme des probabilités (plus on a de mots des lexiques, mieux c'est) et la différence des probabilités (les sites avec un nombre à peu près égal de mots de chaque lexique sont plus intéressants).

La première famille de fonction est :  

$$ score_i = \frac{\lambda*(P_i^I + P_i^G)-\left|{P_i^I-P_i^G}\right|}{2*\lambda}, \forall i \in A, \lambda \in \{10, 5, 2\}$$

avec A l'ensemble des articles et $P_i^I$ et $P_i^G$ respectivement les probabilités d'Innovation et de Gestion pour l'article $i$

La deuxième mesure prise en compte est une mesure se rapprochant de la F1_mesure utilisée pour agréger la précision et le rappel. Cette mesure permet de mettre en avant des probabilités équilibrées. 

$$ F1\_score_i = \frac{2*(P_i^I*P_i^G)}{P_i^I+P_i^G}, \forall i \in A$$ avec A l'ensemble des articles et $P_i^I$ et $P_i^G$ respectivement les probabilités d'Innovation et de Gestion pour l'article $i$


In [25]:
import pandas as pd
def pertinence_6(df : pd.DataFrame): #df avec proba innovation et proba gestion
    """Documentation
    Parameters:
        df : a data frame of scrapped articles with the probability for each to contain a "gestion" word or an "innovation" word (group 5 work)
    Out :
        df : the same data frame with 4 more columns each containing a score based on the probability
    """
    df['pertinence_10'] = df.apply(lambda row: (10*(row.probaInnovation + row.probaGestion) - abs(row.probaInnovation-row.probaGestion))/20, axis = 1) #(10*(motsGestion+motsInnovation)-abs(motsGestion-motsInnovation)) / 20 (pour un score entre 0 et 1)
    df['pertinence_5'] = df.apply(lambda row: (5*(row.probaInnovation + row.probaGestion) - abs(row.probaInnovation-row.probaGestion))/10, axis = 1) #(5*(motsGestion+motsInnovation)-abs(motsGestion-motsInnovation)) / 10 (pour un score entre 0 et 1)
    df['pertinence_2'] = df.apply(lambda row: (2*(row.probaInnovation + row.probaGestion) - abs(row.probaInnovation-row.probaGestion))/4, axis = 1) #(2*(motsGestion+motsInnovation)-abs(motsGestion-motsInnovation)) / 4 (pour un score entre 0 et 1)
    df['pertinence_F1'] = df.apply(lambda row: 2*(row.probaInnovation * row.probaGestion) / (row.probaInnovation+row.probaGestion), axis = 1) #F1 score adapté
    return df



df = pd.DataFrame({'probaInnovation': [0.99,0.8,0.5, 0.2, 0.01]*5, 'probaGestion': [0.99]*5+[0.8]*5+[0.5]*5+[0.2]*5+[0.01]*5})
df1 = pertinence_6(df)
df1.sort_values(by='pertinence_F1', axis=0, ascending=False)


,probaInnovation,probaGestion,pertinence_10,pertinence_5,pertinence_2,pertinence_F1
0,0.99,0.99,0.9900,0.990,0.9900,0.990000
1,0.80,0.99,0.8855,0.876,0.8475,0.884916
5,0.99,0.80,0.8855,0.876,0.8475,0.884916
6,0.80,0.80,0.8000,0.800,0.8000,0.800000
10,0.99,0.50,0.7205,0.696,0.6225,0.664430
2,0.50,0.99,0.7205,0.696,0.6225,0.664430
11,0.80,0.50,0.6350,0.620,0.5750,0.615385
7,0.50,0.80,0.6350,0.620,0.5750,0.615385
12,0.50,0.50,0.5000,0.500,0.5000,0.500000
3,0.20,0.99,0.5555,0.516,0.3975,0.332773


Les résultats précédents montrent bien la pertinence de la fonction proche du F1_score. Cette mesure est donc celle à utiliser pour garder des articles qui parlent conjointement d'innovation et de gestion. 
La famille des mesures permet, quand $\lambda$ augmentent, de mettre en avant des articles ayant un fort score en innovation et un faible score en gestion (ou inversement). 

Les deux méthodes peuvent être utilisées dans notre application. 


In [14]:
def pertinence_6_source(df_pertinence_6 : pd.DataFrame):
    """Documentation
    Parameters: 
        df_pertinence_6 : a data frame of scrapped articles with their score, obtained with the function pertinence_6
    Out :
        df : a data frame of the means of each score for each source
    """
    dico_pertinence : dict = {} #key : a source, values : a list [score_10,score_5,score_2,score_F1,number of articles]
    for articles in df_pertinence_6.itertuples():
        #if the source arlready is in the dictionnary
        if url_site(articles.art_url) in dico_pertinence: 
            #using square function so that articles with a higher score are more important
            dico_pertinence[url_site(articles.art_url)][0] += articles.pertinence_10**2
            dico_pertinence[url_site(articles.art_url)][1] += articles.pertinence_5**2
            dico_pertinence[url_site(articles.art_url)][2] += articles.pertinence_2**2
            dico_pertinence[url_site(articles.art_url)][3] += articles.pertinence_F1**2
            #increasing the number or articles
            dico_pertinence[url_site(articles.art_url)][4] += 1 
        else :
            #adding a new source
            dico_pertinence[url_site(articles.art_url)]=
            [articles.pertinence_10**2,articles.pertinence_5**2,articles.pertinence_2**2,articles.pertinence_F1**2,1]

    df : pd.DataFrame = pd.DataFrame.from_dict(dico_pertinence, orient = 'index', columns = ['pertinence_10','pertinence_5','pertinence_2','pertinence_F1', 'nb_articles']) #turning the dictionnary into a data frame
    df['pertinence_10_moyenne'] = df.apply(lambda row: row.pertinence_10 / row.nb_articles, axis = 1)
    df['pertinence_5_moyenne'] = df.apply(lambda row: row.pertinence_5 / row.nb_articles, axis = 1)
    df['pertinence_2_moyenne'] = df.apply(lambda row: row.pertinence_2 / row.nb_articles, axis = 1)
    df['pertinence_F1_moyenne'] = df.apply(lambda row: row.pertinence_F1 / row.nb_articles, axis = 1)
    return df

La mesure d'agrégation ici est la moyenne par site web. L'utilisation de la fonction carré permet de mettre en avant les articles avec une forte pertinence. 

Cette mesure bien que très pertinentes dépend de deux hypothèses fortes : 
  * le scrapeur générique fonctionne et est performant. 
  * les modèles de classification sont justes.

Cette méthode nécessite donc de scraper les articles visés (avec le scrapeur générique). Ils vont par la suite passer le traitement classique de stockage en Base de Données, faire des embedding pour pouvoir donner une probabilité que l'article parle d'innovation et de gestion. 
Cette méthode peut donc être coûteuse mais permet de donner un bon score de pertinence. 


## Conclusion

Les mesures présentées ci-dessus ne sont pas toutes pertinentes indépendamment les unes des autres, mais peuvent être combinées en les agrégeant (par sites) en faisant par exemple une moyenne pondérée afin de mettre en avant les mesures les plus justes (comme la dernière présentée). 
Dans le cadre de l'automatisation de ce processus seulement les trois premières mesures ont été utilisées.

La liste des mesures n'est pas exhaustive et d'autres peuvent être mises en place.


In [ ]:
def fusion_2(df : pd.DataFrame) :
    """ Documentation

    Calculate the relevance for each site crawled

    Parameters :
        df : dataframe containing the results crawled without the sites we 
             don't want
    Out :
        fusion_result : dataframe containing the score for each site crawled
    """

    df_prepare = prepareDF(df)
    fusion = pd.merge(popularite(df_prepare), relevance_query(df_prepare), how="right", left_on="src_name", right_on="src_name") #the new relevance score is merged with those obtained previously, 
    df_result_rank = score_rank(df_prepare)
    df_result_rank = df_result_rank.reset_index()
    fusion_result = pd.merge(fusion, df_result_rank, how="right", left_on="src_name", right_on="src_name")
    fusion_result['score_mean'] = 0.0
    for i in range (len(fusion_result)):
        fusion_result['score_mean'][i] = fusion_result['popularity'][i]*0.2 + fusion_result['relevance_query'][i]*0.4 + fusion_result['score_rank'][i]*0.4 #These scores are weighted by coefficients applied after reflection and an average relevance score is calculated,
    return(fusion_result)